In [1]:
import re
import subprocess
import os
import json

import pandas as pd

# import xlwings as xw


In [2]:
exec_name = "cyber"
dir = "out/" + exec_name + "/"
dir_md = dir + "comands-md-files/"


In [3]:
if not os.path.isdir(dir):
    os.mkdir(dir)
    os.mkdir(dir_md)

for f in os.listdir(dir_md):
    os.remove(os.path.join(dir_md, f))


In [4]:
chapters_to_include = [
    "Available Commands:",
    "Usage:",
    "Aliases:",
    "Flags:",
    "Global Flags:",
    "Example:",
    "Examples:",
]


In [5]:
def get_cli_output(command: str):

    _args = command.split()
    _args.append("-h")

    out = subprocess.run(_args, capture_output=True)
    text = out.stdout.decode("utf-8") + out.stderr.decode("utf-8")

    with open(dir_md + command + ".md", "w") as file:
        file.write(text)

    return text


# text = get_cli_output('cyber')


In [6]:
def parse_cli_stdout(command: str):

    _text = get_cli_output(command)
    _lines = _text.split("\n")

    chapters = {"command": command}
    chapter = "Description"
    chapters[chapter] = []

    for l in _lines:
        if l != "":
            if l in chapters_to_include:
                chapter = l[:-1]
                chapters[chapter] = []
            else:
                chapters[chapter].append(l)

    return chapters


# chapters = parse_cli_stdout("gaiad")
# print(chapters)


In [7]:
def get_sub_commands(chapters):
    new_commands = []

    if "Available Commands" in chapters:
        for l in chapters["Available Commands"]:
            if _match := re.search(r"^\s+(.*?)\s+(.*)", l):
                sub_command = _match.group(1)

                command_candidate = " ".join(
                    (chapters["command"] + " " + sub_command).split()
                )
                new_commands.append(command_candidate)

    return new_commands
    # del chapters["Available Commands"]


# new_commands = get_sub_commands(chapters)
# new_commands


In [8]:
parsed_commands = []
all_commands = []
new_commands = [exec_name + " "]

while True:
    for command in new_commands:

        commands_dict = parse_cli_stdout(command)
        new_command_candidates = get_sub_commands(commands_dict)
        all_commands.append(commands_dict)

        for candidate in new_command_candidates:
            if candidate not in parsed_commands:
                new_commands.append(candidate)
                parsed_commands.append(candidate)
        new_commands.remove(command)
    if new_commands == []:
        break


In [9]:
# all_commands


In [10]:
json_object = json.dumps(all_commands, indent=4)

with open(dir + exec_name + "_commands.json", "w") as outfile:
    outfile.write(json_object)


In [11]:
len(all_commands)


284

In [12]:
command_files = []

for f in os.listdir(dir_md):
    if f[-3:] == ".md":
        command_files.append(f)

command_files.sort()
# command_files


In [13]:
if os.path.exists(dir + exec_name + "_commands.md"):
    os.remove(dir + exec_name + "_commands.md")

for f in command_files:
    with open(dir_md + f) as f2:
        text = f2.read()
        file_content = f"### {f[:-3]}\n\n```\n{text}\n```\n\n"

        with open(dir + exec_name + "_commands.md", "a") as file:
            file.write(file_content)


In [14]:
# with open("all_commands.json", "r") as file:
#     all_commands = json.load(file)


In [15]:
commands = []

for c in all_commands:
    if "Available Commands" not in c:
        commands.append(c)


In [16]:
count = 0
for i in commands:
    all_flags = []
    all_flags.extend(i["Flags"])
    del i["Flags"]
    if "Global Flags" in i:
        all_flags.extend(i["Global Flags"])
        del i["Global Flags"]

    if all_flags != []:
        i["flags_parsed"] = []
        for b in all_flags:
            if match1 := re.match(
                r"\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[.\w-]+))\s(?:(?P<format>(uint32|ints|strings|int 64Slice|int|string|uint|float|bytesHex)))?\s*(?P<description>.*)?",
                b,
            ):
                i["flags_parsed"].append(
                    {
                        "long": match1.group("long"),
                        "short": match1.group("short"),
                        "format": match1.group("format"),
                        "description": match1.group("description"),
                    }
                )


In [17]:
for c in commands:
    # c["flags_parsed"] = [f for f in c["flags_parsed"] if f["long"] != "help"]
    c["Usage"] = [
        l.replace("[flags]", "").replace(c["command"], "").strip() for l in c["Usage"]
    ]
    c["Description"] = " ".join(c["Description"])[:200]


In [30]:
commands.sort(key=lambda x: x['command'])

In [31]:
# removing optional parameters in brackets ['[name] (--upgrade-height [height]) (--upgrade-info [info])']

for c in commands:
    clean_string = c["Usage"][0].split("(")[0].split("--")[0]
    args = re.findall(r"(?:\[|\<)(\w.+?)(?:\]|\>)", clean_string)
    c["args"] = []
    for p in args:
        p = re.sub(r"[^A-Za-z0-9]", "_", p)
        c["args"].append(p)


In [42]:
commands

[{'command': 'cyber add-genesis-account',
  'Description': 'Add a genesis account to genesis.json. The provided account must specify the account address or key name and a list of initial coins. If a key name is given, the address will be looked up in the local',
  'Usage': ['[address_or_key_name] [coin][,[coin]]'],
  'flags_parsed': [{'long': 'height',
    'short': None,
    'format': 'int',
    'description': 'Use a specific height to query state at (this can error if the node is pruning state)'},
   {'long': 'help',
    'short': 'h',
    'format': None,
    'description': 'help for add-genesis-account'},
   {'long': 'keyring-backend',
    'short': None,
    'format': 'string',
    'description': 'Select keyring\'s backend (os|file|kwallet|pass|test) (default "os")'},
   {'long': 'node',
    'short': None,
    'format': 'string',
    'description': '<host>:<port> to Tendermint RPC interface for this chain (default "tcp://localhost:26657")'},
   {'long': 'output',
    'short': 'o',
   

In [54]:
for c in commands:
    for f in c['flags_parsed']:
        if m:=re.findall(r'(\(.*\|.*?\))', f['description']):
            print(m)

['(os|file|kwallet|pass|test)']
['(text|json)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(sync|async|block)']
['(os|file|kwallet|pass|test|memory)']
['(text|json)']
['(direct|amino-json)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(os|file|test)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(text|json)']
['(os|file|test)']
['(json|plain)']
['(trace|debug|info|warn|error|fatal|panic)']
['(text|json)']
['(os|file|test)']
['(js

In [53]:
re.findall(r'(\(.*\|.*?\))', 'Select keyring\'s backend (os|file|kwallet|pass|test) (default "os")')

['(os|file|kwallet|pass|test)']

In [32]:
# df = pd.DataFrame(commands)
# pd.json_normalize(df['flags_parsed'])


In [33]:
# df = pd.DataFrame(commands).explode('flags_parsed').reset_index()
# df['command'] = df['command'].apply(lambda x: str(x))
# df2 = pd.json_normalize(df['flags_parsed'])

# df3 = pd.concat([df[['command', 'Usage']], df2], axis=1)
# xw.view(df3)


In [34]:
# for_excel = pd.DataFrame(data=commands).applymap(lambda x: str(x))
# xw.view(for_excel)


In [35]:
# for i in all_commands:
#     print(i["command"], "\n", i["flags_parsed"])


In [47]:
types_nu_dict = {
    "uint": "int",
    "uint32": "int",
    # "float": "decimal",
    "float": "string",
    "strings": "string",
    "bytesHex": "string",
    "ints": "string",
}

# nu functions to add at the beginning of the file
functions_list = """def "cy keys table" [] {
	cyber keys list | lines | window 5 -s 5 | 
    each {|it| ($it| parse -r '(?P<col>\w+):(?P<value>.*)')} | 
    each {|it| ($it| transpose -r)} | reduce {|it, acc| $it | append $acc} | 
    select name type address 
}

def "nu-complete cyber keys names" [] {
    (cy keys table).name | zip (cy keys table).address | flatten
  }

"""

lines = [functions_list]

for c in commands:
    lines.append(f"\n# {c['Description']}\nextern '{c['command']}' [\n")
    for p in c["args"]:
        lines.append(f"\t{p}: string\n")
    for f in c["flags_parsed"]:
        if "." not in f["long"]:
            long = f["long"]
            short = "(-" + f["short"] + ")" if f["short"] else ""
            f1 = types_nu_dict.get(f["format"], f["format"])
            format = ": " + f1 if f1 else ""
            description = f["description"]
            lines.append(f"\t--{long}{short}{format}\t\t# {description}\n")
    lines.append("]")

lines = [l.replace("--from: string", '--from: string@"nu-complete cyber keys names"') for l in lines]


with open(dir + exec_name + "_nu_completions.nu", "w") as file:
    file.writelines(lines)
